In [12]:
#points-null-vakues.txt dosyasında;
 # --- ilk kolondaki null değerleri 1 ile 
 # --- ikinci kolondaki null değerleri 0 ile değiştiriniz!! (UserDefinedFunction)
 # --- kısayol
 # --- ilk kolondaki null değerleri ortalama ile
 # --- ikinci kolondaki medyan ile değişitiriniz

from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import IntegerType

spark = SparkSession.Builder().appName('Spark DataFrame Introduction').getOrCreate()
pointsDF = spark.read.option('inferSchema','true')\
                        .option('header','false')\
                        .option('delimiter',',').csv('datasets/points-null-values.txt')
pointsDF = pointsDF.withColumnRenamed('_c0','x')
pointsDF = pointsDF.withColumnRenamed('_c1','y')

#Uzun Yol (Sabit sayı ile değiştirme)
def replaceNullColumn1(x):
    if x is None :
        x = 1
    return x
def replaceNullColumn2(x):
    if x is None :
        x = 0
    return x
replaceUDF1 = UserDefinedFunction(replaceNullColumn1,IntegerType())
replaceUDF2 = UserDefinedFunction(replaceNullColumn2,IntegerType())
pointsDF = pointsDF.withColumn('yeniX',replaceUDF1('x'))
pointsDF = pointsDF.withColumn('yeniY',replaceUDF2('y'))
    
pointsDF.show(90)

+--------+--------+--------+--------+
|       x|       y|   yeniX|   yeniY|
+--------+--------+--------+--------+
|       5|       5|       5|       5|
|     105|     107|     105|     107|
|     105|     106|     105|     106|
|    null|       6|       1|       6|
|       6|       6|       6|       6|
|     106|    null|     106|       0|
|       7|       7|       7|       7|
|    null|       6|       1|       6|
|     107|     107|     107|     107|
|       6|       7|       6|       7|
|       7|    null|       7|       0|
|     108|     108|     108|     108|
|       8|       6|       8|       6|
|    null|     108|       1|     108|
|       5|       8|       5|       8|
|     107|     106|     107|     106|
|       8|       8|       8|       8|
|    null|     107|       1|     107|
|     107|     108|     107|     108|
|     108|     106|     108|     106|
|10000008|10000006|10000008|10000006|
+--------+--------+--------+--------+



In [13]:
#Kısa Yol (Sabit sayı ile değiştirme)
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import IntegerType

spark = SparkSession.Builder().appName('Spark DataFrame Introduction').getOrCreate()
pointsDF = spark.read.option('inferSchema','true')\
                        .option('header','false')\
                        .option('delimiter',',').csv('datasets/points-null-values.txt')
pointsDF = pointsDF.withColumnRenamed('_c0','x')
pointsDF = pointsDF.withColumnRenamed('_c1','y')

pointsDF = pointsDF.fillna(1,'x')
pointsDF = pointsDF.fillna(0,'y')
pointsDF.show()

+---+---+
|  x|  y|
+---+---+
|  5|  5|
|105|107|
|105|106|
|  0|  6|
|  6|  6|
|106|  1|
|  7|  7|
|  0|  6|
|107|107|
|  6|  7|
|  7|  1|
|108|108|
|  8|  6|
|  0|108|
|  5|  8|
|107|106|
|  8|  8|
|  0|107|
|107|108|
|108|106|
+---+---+
only showing top 20 rows



In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import IntegerType

spark = SparkSession.Builder().appName('Spark DataFrame Introduction').getOrCreate()
pointsDF = spark.read.option('inferSchema','true')\
                        .option('header','false')\
                        .option('delimiter',',').csv('datasets/points-null-values.txt')
pointsDF = pointsDF.withColumnRenamed('_c0','x')
pointsDF = pointsDF.withColumnRenamed('_c1','y')
pointsDF.describe().show()

+-------+-----------------+-----------------+
|summary|                x|                y|
+-------+-----------------+-----------------+
|  count|               17|               19|
|   mean|         588289.0|526369.8947368421|
| stddev|2425344.472743975|2294145.690034356|
|    min|                5|                5|
|    max|         10000008|         10000006|
+-------+-----------------+-----------------+



In [21]:
#Kısa Yol (Ortalama ile değiştirme)
from pyspark.sql import SparkSession
from pyspark.ml.feature import Imputer

spark = SparkSession.Builder().appName('Spark DataFrame Introduction').getOrCreate()
pointsDF = spark.read.option('inferSchema','true')\
                        .option('header','false')\
                        .option('delimiter',',').csv('datasets/points-null-values.txt')
pointsDF = pointsDF.withColumnRenamed('_c0','x')
pointsDF = pointsDF.withColumnRenamed('_c1','y')
#pointsDF.show()

#Imputer kullanırken öncelikle outlier var mı diye bakarız
#Eğer outlier yok ise ortalama (Mean) ile değiştirmek mantıklıdır
#Eğer outlier varsa ve silmek istemiyorsak medyan (median) ile değiştirmek mantıklıdır
meanImputer = Imputer(inputCol='x',outputCol='imputedXMean',strategy='mean')
pointsDF = meanImputer.fit(pointsDF).transform(pointsDF)

medianImputer = Imputer(inputCol='y',outputCol='imputedYMedian',strategy='median')
pointsDF = medianImputer.fit(pointsDF).transform(pointsDF)

pointsDF.show(100)

+--------+--------+------------+--------------+
|       x|       y|imputedXMean|imputedYMedian|
+--------+--------+------------+--------------+
|       5|       5|           5|             5|
|     105|     107|         105|           107|
|     105|     106|         105|           106|
|    null|       6|      588289|             6|
|       6|       6|           6|             6|
|     106|    null|         106|           106|
|       7|       7|           7|             7|
|    null|       6|      588289|             6|
|     107|     107|         107|           107|
|       6|       7|           6|             7|
|       7|    null|           7|           106|
|     108|     108|         108|           108|
|       8|       6|           8|             6|
|    null|     108|      588289|           108|
|       5|       8|           5|             8|
|     107|     106|         107|           106|
|       8|       8|           8|             8|
|    null|     107|      588289|        

In [ ]:
#Homework 2 hint: DataFrame pivot, Recommendation Engine , Collabrative Filtering